In [1]:
import plotly.express as px
import plotly.graph_objects as go
import os
import pandas as pd
import numpy as np 
import s3fs
import configparser

config = configparser.ConfigParser()
# Este archivo se creará cuando se instale el AWS command line interface.
# Seguir estas instrucciones: https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html
# ES OBLIGATORIO instalar este componente antes de continuar esta ejecución.
config.read(os.getenv("HOME") + '/.aws/credentials')

aws_credentials = {'key': config["cdas-itam"]["aws_access_key_id"],'secret': config["cdas-itam"]["aws_secret_access_key"]}

df = pd.read_csv(r's3://captain-planet-denuncias-paot/clean/clean_denuncias_paot.csv', storage_options=aws_credentials, low_memory=False)

df[["fecha_de_recepcion", "fecha_de_ratificacion", "fecha_de_turno", "fecha_de_admision/radicacion", "fecha_de_conclusion"]] = df[["fecha_de_recepcion", "fecha_de_ratificacion", "fecha_de_turno", "fecha_de_admision/radicacion", "fecha_de_conclusion"]].apply(pd.to_datetime, errors = 'coerce', infer_datetime_format=True)


In [2]:


import pandas as pd

from simpledbf import Dbf5

dbf = Dbf5(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\raw\ri_11.dbf')

df_pob = dbf.to_dataframe()



def correlacion_densidad_parametro(columna, parametro):

    #dataframe vacio
    df_comparativo = pd.DataFrame(columns=['colonia', 'num_denuncias','hab_por_km_cuadrado'], index = range(df['colonia'].unique().size))

    #iniciar indice
    index = 0
    
    for colonia in df['colonia'].unique():
        

        #checar q la colina exista en la base de densidad
        if(colonia.upper() in df_pob['colonia'].unique()):

            #asignar colonia
            df_comparativo.iloc[index, 0] = colonia

            #numero de denuncias por colonia siguiendo parametro de columna dados
            df_comparativo.iloc[index,1] =  df[(df[columna]==parametro) & (df['colonia']==colonia)].shape[0]
            
            #& (df['fecha_de_recepcion'].dt.year == 2010) se reducen demasiado los resultados

            #datos especificos de la colonia (en tabla densidad)
            datos_colonia = df_pob[df_pob['colonia'] == colonia.upper()]

            if(not datos_colonia.empty):
                #densidad x km cuadrado
                df_comparativo.iloc[index,2] = (1000000/int(datos_colonia.iloc[0,5])*int(datos_colonia.iloc[0,4]))
            

      
        
        index = index +1 
        
        

    return df_comparativo.dropna()


correlacion_densidad_parametro('materia', 'Maltrato Animal')




PyTables is not installed. No support for HDF output.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\samue\\OneDrive\\Documents\\GitHub\\captain_planet_sammy\\captain-planet\\data\\raw\\ri_11.dbf'

Pruebas iniciales de mergear datasets, sin linkage

In [ ]:
df_pob[df_pob['colonia']=='SAN RAFAEL']

In [ ]:
correlacion_materias = pd.DataFrame(columns=['materia','correlacion_con_densidad'], index = range(df['materia'].unique().size))

index = 0
for materia in df['materia'].unique():
    tabla_denuncias_densidad = correlacion_densidad_parametro('materia', materia)
    correlacion_materias.iloc[index, 0] = materia
    correlacion_materias.iloc[index, 1] = tabla_denuncias_densidad['num_denuncias'].astype('float64').corr(tabla_denuncias_densidad['hab_por_km_cuadrado'].astype('float64'))
    index = index + 1


# se logro :) la tabla con correlaciones de materia con densidad de poblacion
correlacion_materias.head(50)



In [ ]:
correlacion_materias.to_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\clean\correlacion_materia_densidad.csv', encoding = 'latin')

In [ ]:
#para presentarlo al equipo

correlacion_materias = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\clean\correlacion_materia_densidad.csv', encoding = 'latin', index_col= 0)
correlacion_materias